# Imports

In [53]:
# IMPORT
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.path import Path
import numpy as np
import seaborn as sns

# bokeh
import bokeh.palettes
from bokeh.plotting import figure, show, output_file, save, reset_output
from bokeh.io import output_notebook
from bokeh.models import HoverTool, Legend, ColumnDataSource, Title
from bokeh.tile_providers import get_provider, CARTODBPOSITRON
from bokeh.models import GeoJSONDataSource
from bokeh.layouts import layout



output_notebook()
np.random.seed(42)

Loading BokehJS ...

# Constants

In [21]:
def get_color_from_palette(color):
    """ Getting colors for plotting """
    return tuple([int(c * 255) for c in color])

def get_dark_color_from_palette(color):
    """ Getting darker colors for plotting """
    return tuple([int(c * 200) for c in color])

PALETTE = "colorblind"

In [48]:
sns.color_palette(PALETTE)

[(0.00392156862745098, 0.45098039215686275, 0.6980392156862745),
 (0.8705882352941177, 0.5607843137254902, 0.0196078431372549),
 (0.00784313725490196, 0.6196078431372549, 0.45098039215686275),
 (0.8352941176470589, 0.3686274509803922, 0.0),
 (0.8, 0.47058823529411764, 0.7372549019607844),
 (0.792156862745098, 0.5686274509803921, 0.3803921568627451),
 (0.984313725490196, 0.6862745098039216, 0.8941176470588236),
 (0.5803921568627451, 0.5803921568627451, 0.5803921568627451),
 (0.9254901960784314, 0.8823529411764706, 0.2),
 (0.33725490196078434, 0.7058823529411765, 0.9137254901960784)]

In [22]:
district_colors = [
    get_color_from_palette(c)
    for c in sns.color_palette(PALETTE, 21)
    ]

district_dark_colors = [
    get_dark_color_from_palette(c)
    for c in sns.color_palette(PALETTE, 21)
    ]

madrid_in_out_colors = [
    get_color_from_palette(c)
    for c in sns.color_palette(PALETTE, 2)
    ]

madrid_in_out_dark_colors = [
    get_dark_color_from_palette(c)
    for c in sns.color_palette(PALETTE, 2)
    ]

# Madrid Central

We are going to analyze the impact of Madrid Central both from an air quality and a traffic viewpoint.

## Air Quality

In [5]:
# load air quality stations
df_stations = pd.read_csv('shared_data/air_quality/air_quality_stations.csv')

# load magnitud table
df_magnitud = pd.read_csv('shared_data/air_quality/air_quality_magnitud.csv', sep=';')

# load air quality data
df = pd.read_csv('data/air_quality_data.csv')

# converting Date to datetime type
df["datetime"] = pd.to_datetime(df["datetime"])

# merge with air quality stations
df = pd.merge(df, df_stations, left_on = 'PUNTO_MUESTREO', right_on='punto_muestreo', how='left').drop('PUNTO_MUESTREO', axis=1)

# merge with air quality magnitud
df = pd.merge(df, df_magnitud, left_on = 'MAGNITUD', right_on='magnitud_id', how='left').drop('MAGNITUD', axis=1)

df.head()

,PROVINCIA,MUNICIPIO,ESTACION,value,datetime,punto_muestreo,name,longitude,latitude,altitude,utm_x,utm_y,magnitud_id,formula,unit_per_m3
0,28,79,4,7.0,2016-04-01 01:00:00,28079004,Plaza de España,-3.712197,40.423883,637,-413239.904502,4.927732e+06,1,SO2,µg
1,28,79,4,8.0,2016-04-02 01:00:00,28079004,Plaza de España,-3.712197,40.423883,637,-413239.904502,4.927732e+06,1,SO2,µg
2,28,79,4,10.0,2016-04-03 01:00:00,28079004,Plaza de España,-3.712197,40.423883,637,-413239.904502,4.927732e+06,1,SO2,µg
3,28,79,4,7.0,2016-04-04 01:00:00,28079004,Plaza de España,-3.712197,40.423883,637,-413239.904502,4.927732e+06,1,SO2,µg
4,28,79,4,8.0,2016-04-05 01:00:00,28079004,Plaza de España,-3.712197,40.423883,637,-413239.904502,4.927732e+06,1,SO2,µg


In [8]:
print('AIR QUALITY STATION NAMES:')
print([name for name in df.name.unique()])

AIR QUALITY STATION NAMES:
['Plaza de España', 'Escuelas Aguirre', 'Ramón y Cajal', 'Arturo Soria', 'Villaverde', 'Farolillo', 'Casa de Campo', 'Barajas Pueblo', 'Plaza del Carmen', 'Moratalaz', 'Cuatro Caminos', 'Barrio del Pilar', 'Vallecas', 'Méndez Álvaro', 'Castellana', 'Retiro', 'Plaza Castilla', 'Ensanche de Vallecas', 'Urbanización Embajada', 'Plaza Elíptica', 'Sanchinarro', 'El Pardo', 'Juan Carlos I', 'Tres Olivos']


In [28]:
# MAP
# load MC area
cm_points = pd.read_csv('shared_data/districts/central_madrid_points.csv')

points = df_stations[["utm_x", "utm_y"]].values
path = Path(cm_points[["utm_x", "utm_y"]].values)
points_in_path_mask = path.contains_points(points)

df_stations["madrid_central"] = False

df_stations.loc[points_in_path_mask, "madrid_central"] = True

df_stations.head()

,punto_muestreo,name,longitude,latitude,altitude,utm_x,utm_y,madrid_central
0,28079035,Plaza del Carmen,-3.703167,40.419208,660,-412234.627656,4.927049e+06,True
1,28079004,Plaza de España,-3.712197,40.423883,637,-413239.904502,4.927732e+06,False
2,28079039,Barrio del Pilar,-3.711536,40.478233,676,-413166.309950,4.935683e+06,False
3,28079008,Escuelas Aguirre,-3.682283,40.421564,672,-409909.905623,4.927393e+06,False
4,28079038,Cuatro Caminos,-3.707122,40.445547,699,-412674.958086,4.930901e+06,False


In [54]:
# plot map
p = figure(title="Air quality stations in Madrid", x_axis_type="mercator", y_axis_type="mercator")

source_mc = ColumnDataSource(cm_points)
p.line(x="utm_x", y="utm_y", color=madrid_in_out_colors[0], line_width=4, 
        source=source_mc, legend_label="Madrid Central limit", muted=False, muted_alpha=0.3)

source_in = ColumnDataSource(df_stations[df_stations["madrid_central"] == True])

source_out = ColumnDataSource(df_stations[df_stations["madrid_central"] == False])

cr_in = p.circle(x="utm_x", y="utm_y",  size=20, source=source_in,
                color=madrid_in_out_colors[0],line_width=1, line_color=madrid_in_out_dark_colors[0],
                legend_label="IN Madrid Central area")

cr_out = p.triangle(x="utm_x", y="utm_y",  size=20, source=source_out,
                color=madrid_in_out_colors[1], line_width=1, line_color=madrid_in_out_dark_colors[1],
                legend_label="OUT of Madrid Central area")

cartodb = get_provider(CARTODBPOSITRON)
p.add_tile(cartodb)

p.add_tools(HoverTool(tooltips=[('Name', '@name')], renderers=[cr_in, cr_out]))

# add interactive legend
# legend = Legend(items=[(, [cr_in]), ('OUT of Madrid Central area', [cr_out])], location='center') 
p.legend.click_policy="hide"
# legend.location = "top_left"

p = layout(p, sizing_mode='scale_both')

output_file("html_plots/air_quality_stations.html", title="Air quality stations")
save(p)
reset_output()
output_notebook()

# show(p)

Loading BokehJS ...